In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

sns.set(style='whitegrid')
sys.path.append('..')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=RuntimeWarning)

from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression, LogisticRegression
from load_data import *

from numpy.random import default_rng
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.optimize import curve_fit

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

<br><br>

# Functions

In [2]:
def extract_part_test_data(dataframe_test, create_amount_of_random_numbers = 6, distance_of_max_length = 0):
    """
    This function takes as input the df_test and creates for each engine id
    random amount of numbers (create_amount_of_random_numbers). Then it sorts 
    those numbers and takes the lowest and biggest number (e.g. 4 and 142). It
    uses those numbers to slice a random part.
    
    We also can add the parameter 'distance_of_max_length'. This parameter allows
    us to for example set a limit of -50 of the maximum length of an engine id. If
    the engine_id has e.g. length 230 (where the RUL is 0), we can set the limit to
    finding random number between 0 and 180 (instead of 230).
    """
    # Extract all the unique engine ids and their maximum length in the original dataframe
    unique_engine_ids = dataframe_test.index.unique()
    amount_of_ruls    = {engine_id : dataframe_test[dataframe_test.index == engine_id]['cycle'].max()
                         for engine_id in unique_engine_ids}
    
    
    # Loop over all engine ids and their length in the original dataframe_test
    print('The test dataframe includes the following engine ids: {}.\n'.format(list(unique_engine_ids)))
    print('We sliced the test dataframe into the following slices: \n')
    new_df = pd.DataFrame()
    
    for engine_id, max_length in amount_of_ruls.items():
        random_numbers = np.random.randint(0, (max_length - 1 - distance_of_max_length), create_amount_of_random_numbers)
        random_numbers_sorted = np.sort(random_numbers, axis=None)
        print('Engine Id {} filtered on index : {}.'.format(engine_id, [random_numbers_sorted[0], random_numbers_sorted[-1]]))
        print('This slice has a length of {}.'.format(random_numbers_sorted[-1] - random_numbers_sorted[0]))
        print('The original total length of the engine id was {}.'.format(amount_of_ruls[engine_id]))
        print('------------------------------------------------\n')
        
        df_filtered = dataframe_test[dataframe_test.index == engine_id]
        df_filtered = df_filtered.iloc[random_numbers_sorted[0] : random_numbers_sorted[-1]]
        
        new_df = new_df.append(df_filtered)
        
    return new_df

In [3]:
def get_ruls(dataframe):
    """
    Get the ruls of the engines
    """
    # Try to drop max cycle to prevent an error
    dataframe = dataframe.drop('max_cycle', 1, errors='ignore')
    
    # The max cycle of each engine
    max_cycle = dataframe.groupby('engine_id').agg({'cycle': 'max'}).rename({'cycle': 'max_cycle'}, axis=1)
    
    # Add the max cycle to the dataframe
    dataframe = dataframe.join(max_cycle, on='engine_id', )
    
    # Ruls are max cycle minus current cycle (plus 1)
    ruls = (dataframe['max_cycle'] - dataframe['cycle'] + 1).values
    
    return ruls

In [4]:
def load_add_rul_filter_and_norm_data(percentage_test_set, path = '../data/DataTrain.txt'):
    df = load_dataset(path)
    df = df.set_index('engine_id')
    
    # Based on the analysing the slopes we found that the following sensors had
    # showed a trend and had the highest average slope. We focus on the sensors 
    # with higher slopes because this makes it easier to distinguish the different RULs.
    df = df[['cycle', 's4', 's3', 's17', 's7', 's12', 's2']]
    df.loc[:, 'rul'] = get_ruls(df)
    
    # Calculate percentage test set
    unique_engine_ids = df.index.unique()
    
    engine_ids_for_test_set = default_rng().choice(unique_engine_ids,
                                                   size = int(max(unique_engine_ids) * percentage_test_set), 
                                                   replace=False)
    
    # Split data into df_train and df_test
    df_train = df[~df.index.isin(engine_ids_for_test_set)]
    df_test  = df[df.index.isin(engine_ids_for_test_set)]
    
    # Normalize both dataframes
    scaler = StandardScaler()
    df_train.iloc[:, 1:-1] = scaler.fit_transform(df_train.iloc[:, 1:-1])
    df_test.iloc[:, 1:-1]  = scaler.transform(df_test.iloc[:, 1:-1])
    
    return df_train, df_test

In [5]:
def slopes_all_engine(training_dataframe):
    """
    Get the sensor slopes of all engines.
    """
    def get_training_data(dataframe, engine_id, sensor_cols):
        # X data
        X = dataframe.loc[engine_id, 'rul'].values
        X = X.reshape(-1, 1)

        # y data
        y = dataframe.loc[engine_id, sensor_cols].values

        return (X, y)
    
    def slopes_of_engine(X, y):
        """
        Fit a linear model on the sensor values of one engine
        """
        # Create a linear model
        linear_model = LinearRegression()

        # Fit the model
        linear_model.fit(X, y)

        # Get slopes
        slopes = linear_model.coef_[:,0]

        return slopes
    
    # Loop over all the engines and calculate the slopes.
    engine_slopes = {}
    sensor_cols = training_dataframe.columns[1: -1].tolist()
    
    for engine_id in training_dataframe.index.unique():
        X, y = get_training_data(training_dataframe, engine_id, sensor_cols)
        slopes = slopes_of_engine(X, y)
        
        engine_slopes[engine_id] = slopes
        
    df_engine_slopes = pd.DataFrame(engine_slopes).T

    df_engine_slopes.columns = sensor_cols
    df_engine_slopes.index.name = 'engine_id'

    return df_engine_slopes

In [6]:
def perform_pca(dataframe_with_engine_slopes, dataframe_training, dataframe_testing, 
                n_components = 3, keep_rul_in_test = True):
    """
    Here we will perform the PCA on the training and test set.
    """
    # Fit the PCA on the engine slopes to reduce the dimensionality.
    pca = PCA(n_components=n_components)
    pca.fit(dataframe_with_engine_slopes)
        
    # Apply PCA on the dataframes    
    matrix_training = np.column_stack([dataframe_training[['cycle', 'rul']].values, pca.transform(dataframe_training.iloc[:, 1:-1])])   
    df_pca_training = pd.DataFrame(matrix_training, index=dataframe_training.index, columns=['cycle', 'rul', 'pca1', 'pca2', 'pca3'])
        
    if keep_rul_in_test == True:
        matrix_testing  = np.column_stack([dataframe_testing[['cycle', 'rul']].values, pca.transform(dataframe_testing.iloc[:, 1:-1])])
        df_pca_testing  = pd.DataFrame(matrix_testing, index=dataframe_testing.index, columns=['cycle', 'rul', 'pca1', 'pca2', 'pca3'])

    else: 
        matrix_testing  = np.column_stack([dataframe_testing[['cycle']].values, pca.transform(dataframe_testing.iloc[:, 1:])])
        df_pca_testing  = pd.DataFrame(matrix_testing, index=dataframe_testing.index, columns=['cycle', 'pca1', 'pca2', 'pca3'])

    return df_pca_training, df_pca_testing

In [7]:
def train_health_indictor_model(dataframe_training_with_pca_values, high_rul = 250, low_rul = 5):
    """
    This function applies sensor fusing to the training data and returns a
    trained model to make predictions.
    """
    # Extract ruls
    ruls     = dataframe_training_with_pca_values['rul'].values
    
    # Extract rows
    idx_high_health = [ruls > high_rul][0]
    idx_low_health  = [ruls <= low_rul][0]
    
    # PCA to perform sensor fusing on
    high_health_data = dataframe_training_with_pca_values.loc[idx_high_health, ['pca1', 'pca2', 'pca3']]
    low_health_data  = dataframe_training_with_pca_values.loc[idx_low_health, ['pca1', 'pca2', 'pca3']]
    
    # concatenate high HI and Low HI data
    X_health = np.concatenate((high_health_data, low_health_data),axis=0)

    # target for the fused signal [ just 0 or 1 for failed ans healthy]
    y_one = np.ones(high_health_data.shape[0])
    y_zero = np.zeros(low_health_data.shape[0])

    # concatenate high HI and Low HI target
    y_health = np.concatenate((y_one, y_zero),axis=0)
    
    # Linear regression
    linear_model = LinearRegression()
    linear_model.fit(X_health, y_health)
    
    return linear_model

In [8]:
def add_exponential_fits(dataframe_with_health_index):
    def exp_func(x, a, b):
        return a * (np.exp(b * -x)-1)

    def find_exp_params(dataframe):
        exp_params = np.zeros((dataframe.index.nunique(), 2))

        # Loop through the engines
        for idx, (engine_id, engine) in enumerate(dataframe.groupby('engine_id')):
            x = engine['rul']
            y = savgol_filter(engine['hi_pred'], 25, 1)

            popt, _ = curve_fit(exp_func, x, y)

            exp_params[idx, :] = popt

        return exp_params

    def get_exponential(timestep, row):
        return exp_func(timestep['rul'], *exp_params[row, :])

    exp_params = find_exp_params(dataframe_with_health_index)

    dataframe_with_health_index['exp'] = 0

    for index, engine_id in enumerate(dataframe_with_health_index.index.unique()):
        indexes = dataframe_with_health_index[dataframe_with_health_index.index == engine_id].index
        dataframe_with_health_index.loc[indexes, 'exp'] = dataframe_with_health_index.loc[indexes].apply(lambda x: get_exponential(x, index), axis =1)
    
    return dataframe_with_health_index

In [82]:
def predict_rul_based_on_weighted_ssd(list_with_ssd_and_rul, extract_best_x = 5):
    """
    This function makes based on a nested list [(ssd1, rul1), ... , (ssdN, rulN)]
    a weighted prediction based on the ssd.
    """
    
    # Create a dataframe of the nested list
    df_ssd_and_rul = pd.DataFrame(list_with_ssd_and_rul, columns=['ssd', 'rul'])
        
    # Sort the dataframe based on the ssd and extract top X (extract_best_x)
    df_ssd_and_rul = df_ssd_and_rul.sort_values('ssd').iloc[ : extract_best_x]
    
    # Add percentage ssd of total
    sum_ssd = sum(1 / df_ssd_and_rul['ssd'].values)   
    df_ssd_and_rul['percentage_of_total'] = (1 / df_ssd_and_rul['ssd']) / sum_ssd
    
    # Make weighted predictions
    weighted_prediction = sum(df_ssd_and_rul['rul'] * df_ssd_and_rul['percentage_of_total'])
    
    # Return prediction
    return weighted_prediction

In [74]:
def find_for_engineid_lowest_loss(dataframe_training, 
                                  dataframe_testing_engineid,
                                  make_prediction_only = False):
    
    results = list()
    length    = len(dataframe_testing_engineid)
    
    # We will look in each engine (of dataframe_training) for the lowest loss.
    # When we find it, we will store it with the predicted RUL. This allows us 
    # to make predictions taking into account all the engineids.
    for engine_id in dataframe_training.index.unique():
        filtered_df = dataframe_training[dataframe_training.index == engine_id]
        best_loss   = np.inf
        best_train_slice = list()
        
        # Loop over each array in the filtered dataframe and try to find the 
        # best 'exp' where the loss is the lowest.
        for row_index in range(0, len(filtered_df) - length):
            train_slice = filtered_df.iloc[row_index : (row_index+length)]
            train_health_index = train_slice['exp'].values
            test_health_index  = dataframe_testing_engineid['hi_pred'].values

            loss = np.sum((train_health_index - test_health_index) ** 2)

            if loss < best_loss:
                best_loss        = loss
                best_train_slice = train_slice
                
        # If we couldn't find any good fit on the 'exp' in this engine id, skip the
        # appending to the results.
        if best_loss == np.inf:
            continue
            
        # Add the result (loss and the RUL)
        results.append([best_loss, int(best_train_slice.iloc[-1]['rul'])])
        
    # Make a weighted prediction based on the nested list
    weighted_prediction = predict_rul_based_on_weighted_ssd(results)
    
    if make_prediction_only == False:
        best_rul_difference = abs(int(dataframe_testing_engineid.iloc[-1]['rul']) - weighted_prediction)

        print('Best loss:      ', best_loss)
        print('Train RUL:      ', int(weighted_prediction))
        print('Test RUL:       ', int(dataframe_testing_engineid.iloc[-1]['rul']))
        print('RUL difference: ', best_rul_difference)
        print('\n')    
            
        return best_rul_difference
    
    else:
        return weighted_prediction

In [75]:
def final_prediction(dataframe_training, dataframe_schedule):
    """
    This function makes a prediction on the dataframe_schedule using the
    dataframe_training.
    
    To make a prediction, we apply the following steps:
    1. Extract the relevant columns from both tables.
    2. Calculate the slopes of the dataframe_training.
    3. Apply PCA and transform both dataframe_training and dataframe_schedule.
    4. Train a linear model to predict the health indicators.
    5. Predict the health indicators (hi_pred) of both tables.
    6. Add fit an exponential model on the hi_pred of dataframe_training (exp).
    7. Try to fit each engine_ids hi_pred in the dataframe_schedule on the best
       suiting 'exp'. Based on the best fit we will predict the RUL.
    """
    # Extract the relevant columns from both tables
    columns = ['cycle', 's4', 's3', 's17', 's7', 's12', 's2', 'rul']
    
    dataframe_training = dataframe_training[columns]
    dataframe_schedule = dataframe_schedule[columns[: -1]]
            
    # Calculate slopes of the dataframe_training
    dataframe_training_slopes = slopes_all_engine(dataframe_training)
    
    # Reduce dimensionalities with PCA
    dataframe_training_pca, dataframe_schedule_pca = perform_pca(dataframe_training_slopes, 
                                                                 dataframe_training, 
                                                                 dataframe_schedule, 
                                                                 keep_rul_in_test = False)
    
    # Train linear model
    trained_model = train_health_indictor_model(dataframe_training_pca)
        
    # Predict health indicator
    dataframe_training['hi_pred'] = trained_model.predict(dataframe_training_pca[['pca1', 'pca2', 'pca3']])
    dataframe_schedule['hi_pred'] = trained_model.predict(dataframe_schedule_pca[['pca1', 'pca2', 'pca3']])
    
    # Add exponential fit
    dataframe_training = add_exponential_fits(dataframe_training)
    
    # Make predictions
    results = {}
    
    for engineid in dataframe_training.index.unique():
        results[engineid] = find_for_engineid_lowest_loss(dataframe_training, 
                                                          dataframe_schedule[dataframe_schedule.index == engineid],
                                                          make_prediction_only = True)    
        
    return results

<br><br><br>

# Validation set

In [19]:
%%time

# Create split
df_training, df_testing = load_add_rul_filter_and_norm_data(percentage_test_set=0.2)

# Extract random slice of dataframe
# df_testing = extract_part_test_data(df_testing, create_amount_of_random_numbers = 6, distance_of_max_length = 50)

# Extract slopes
df_slopes = slopes_all_engine(df_training)

# Reduce the dimensionalities to 3 dimensions
df_training_pca, df_testing_pca = perform_pca(df_slopes, df_training, df_testing, keep_rul_in_test = True)

# Train model to predict Health Indicator
gekke_model = train_health_indictor_model(df_training_pca)

# Add health indicator
df_training_pca['hi_pred'] = gekke_model.predict(df_training_pca[['pca1', 'pca2', 'pca3']])
df_testing_pca['hi_pred']  = gekke_model.predict(df_testing_pca[['pca1', 'pca2', 'pca3']])

Wall time: 259 ms


In [20]:
%%time
# Add exp model
df_training_with_hi_and_exp = add_exponential_fits(df_training_pca)

Wall time: 1min 39s


In [21]:
%%time

total_rul_diff = 0

for engineid in df_testing_pca.index.unique():
    rul_difference = find_for_engineid_lowest_loss(df_training_with_hi_and_exp, 
                                                   df_testing_pca[df_testing_pca.index == engineid])
    print('\n\n\nEngine id {} rul difference: {}.\n\n\n'.format(engineid, rul_difference))
    
    
    total_rul_diff += rul_difference
    
total_rul_diff

Best loss:       inf
Train RUL:       14
Test RUL:        1
RUL difference:  13.56544729613006





Engine id 5 rul difference: 13.56544729613006.



Best loss:       inf
Train RUL:       13
Test RUL:        1
RUL difference:  12.088934918717218





Engine id 7 rul difference: 12.088934918717218.



Best loss:       inf
Train RUL:       20
Test RUL:        1
RUL difference:  19.374095857036828





Engine id 10 rul difference: 19.374095857036828.



Best loss:       1.3979587747797342
Train RUL:       7
Test RUL:        1
RUL difference:  6.7775423082784325





Engine id 14 rul difference: 6.7775423082784325.



Best loss:       inf
Train RUL:       1
Test RUL:        1
RUL difference:  0.9999999999999996





Engine id 16 rul difference: 0.9999999999999996.



Best loss:       inf
Train RUL:       4
Test RUL:        1
RUL difference:  3.90970703294452





Engine id 25 rul difference: 3.90970703294452.



Best loss:       2.946788379040118
Train RUL:       12
Test RUL:        1
RUL 

141.59211006596712

<br><br><br><br><br>

In [22]:
%%time

# Create split
df_training, df_testing = load_add_rul_filter_and_norm_data(percentage_test_set=0.2)

# Extract random slice of dataframe
df_testing = extract_part_test_data(df_testing, create_amount_of_random_numbers = 6, distance_of_max_length = 50)

# Extract slopes
df_slopes = slopes_all_engine(df_training)

# Reduce the dimensionalities to 3 dimensions
df_training_pca, df_testing_pca = perform_pca(df_slopes, df_training, df_testing, keep_rul_in_test = True)

# Train model to predict Health Indicator
gekke_model = train_health_indictor_model(df_training_pca)

# Add health indicator
df_training_pca['hi_pred'] = gekke_model.predict(df_training_pca[['pca1', 'pca2', 'pca3']])
df_testing_pca['hi_pred']  = gekke_model.predict(df_testing_pca[['pca1', 'pca2', 'pca3']])

The test dataframe includes the following engine ids: [6, 10, 12, 14, 15, 19, 24, 36, 41, 59, 60, 63, 64, 67, 75, 78, 82, 96, 97, 99].

We sliced the test dataframe into the following slices: 

Engine Id 6 filtered on index : [13, 84].
This slice has a length of 71.
The original total length of the engine id was 188.
------------------------------------------------

Engine Id 10 filtered on index : [10, 164].
This slice has a length of 154.
The original total length of the engine id was 222.
------------------------------------------------

Engine Id 12 filtered on index : [23, 104].
This slice has a length of 81.
The original total length of the engine id was 170.
------------------------------------------------

Engine Id 14 filtered on index : [13, 111].
This slice has a length of 98.
The original total length of the engine id was 180.
------------------------------------------------

Engine Id 15 filtered on index : [7, 124].
This slice has a length of 117.
The original total lengt

In [23]:
# Add exp model
df_training_with_hi_and_exp = add_exponential_fits(df_training_pca)

In [83]:
%%time

total_rul_diff = 0

for engineid in df_testing_pca.index.unique():
    rul_difference = find_for_engineid_lowest_loss(df_training_with_hi_and_exp, 
                                                   df_testing_pca[df_testing_pca.index == engineid])
    print('\n\n\nEngine id {} rul difference: {}.\n\n\n'.format(engineid, rul_difference))
    
    
    total_rul_diff += rul_difference
    
total_rul_diff

Best loss:       0.3564419177068627
Train RUL:       91
Test RUL:        105
RUL difference:  13.932144743804471





Engine id 6 rul difference: 13.932144743804471.



Best loss:       9.6451814404716
Train RUL:       52
Test RUL:        59
RUL difference:  6.850879786596188





Engine id 10 rul difference: 6.850879786596188.



Best loss:       0.5212735239348429
Train RUL:       84
Test RUL:        67
RUL difference:  17.24065659919313





Engine id 12 rul difference: 17.24065659919313.



Best loss:       0.5285684919135368
Train RUL:       63
Test RUL:        70
RUL difference:  6.736305536564075





Engine id 14 rul difference: 6.736305536564075.



Best loss:       6.858703190616704
Train RUL:       90
Test RUL:        84
RUL difference:  6.040848898418062





Engine id 15 rul difference: 6.040848898418062.



Best loss:       0.3159980862487085
Train RUL:       61
Test RUL:        53
RUL difference:  8.001808942090427





Engine id 19 rul difference: 8.001808942090427.





327.1825060136843

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Final prediction

In [85]:
%%time

df_train               = load_dataset('../data/DataTrain.txt')
df_train               = df_train.set_index('engine_id')
df_train.loc[: ,'rul'] = get_ruls(df_train)

df_schedule = load_dataset('../data/DataSchedule.txt')
df_schedule = df_schedule.set_index('engine_id')

predictions = final_prediction(df_train, df_schedule)

KeyboardInterrupt: 

In [ ]:
predictions